## Etapa 1: Importação de Bibliotecas Necessárias

In [1]:
import pandas as pd
import gdown
import plotly.express as px
import plotly.graph_objects as go
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight


## Etapa 2: Carregamento de Dados

In [2]:
# Base do nome do arquivo destino
arquivo_destino_base = "dataset_{}.csv"

# IDs dos arquivos no Google Drive
ids = {
    "dados": "1oEXsdLnaLxsnf7LURAdgAy5ZjICFuQGv",
}

# Dicionário para armazenar DataFrames
dataframes = {}

# Loop para baixar e ler cada arquivo
for key, file_id in ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    arquivo_destino = arquivo_destino_base.format(key)

    # Baixa o arquivo usando gdown
    gdown.download(url, arquivo_destino, quiet=False)

    # Tenta ler o arquivo com pandas
    try:
        df = pd.read_csv(arquivo_destino)
        dataframes[key] = df
    except pd.errors.ParserError:
        print(f"Erro ao ler o arquivo {arquivo_destino}. Verifique o separador.")

Downloading...
From (original): https://drive.google.com/uc?id=1oEXsdLnaLxsnf7LURAdgAy5ZjICFuQGv
From (redirected): https://drive.google.com/uc?id=1oEXsdLnaLxsnf7LURAdgAy5ZjICFuQGv&confirm=t&uuid=e8c4dc9e-2f7e-415a-97b8-4b85391af637
To: /content/dataset_dados.csv
100%|██████████| 683M/683M [00:11<00:00, 58.2MB/s]


### Utilizando apenas 50% dos dados
- sample(frac=0.5): Seleciona 50% das linhas de forma aleatória.
- random_state=42: Garante que a seleção seja replicável. Isso significa que, se você rodar o código novamente, obterá a mesma amostra.


### Por que usar amostragem?
- Redução no uso de memória: Carregar 30% dos dados reduz a RAM necessária, permitindo que prosseguimos com o fluxo de análise, mesmo com dados grandes.
- Impacto nas métricas: As métricas podem ser ligeiramente diferentes, mas a análise ainda será válida.

In [3]:
# Carregar o dataset completo
dados = pd.read_csv("/content/dataset_dados.csv", delimiter=",")

# Selecionar uma amostra de 50% dos dados
dados_amostra = dados.sample(frac=0.5, random_state=42)

# Exibir a amostra para verificação
display(dados_amostra)

# Verificar a nova dimensão do dataset após a amostragem
print(f"Dimensão original: {dados.shape}")
print(f"Dimensão após a amostragem: {dados_amostra.shape}")


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
470624,3457624,0,12153579,724.000,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
565820,3552820,0,15005886,108.500,W,12544,321.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284083,3271083,0,6970178,47.950,W,9400,111.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239689,3226689,0,5673658,100.599,C,15885,545.0,185.0,visa,138.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
281855,3268855,0,6886780,107.950,W,15497,490.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583468,3570468,0,15606925,57.950,W,5017,432.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249281,3236281,0,5943366,57.950,W,9500,321.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87138,3074138,0,1828390,250.000,R,2538,476.0,150.0,visa,166.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
142863,3129863,0,2934013,150.000,R,4425,562.0,150.0,mastercard,197.0,...,1.0,150.0,150.0,150.0,0.0,0.0,0.0,150.0,150.0,150.0


Dimensão original: (590540, 394)
Dimensão após a amostragem: (295270, 394)


##Etapa 3: Análise Exploratória dos Dados (EDA)
Nessa etapa realizamos uma análise exploratória detalhada para entender as distribuições, detectar outliers e identificar possíveis padrões nas transações fraudulentas. Para isso, foram incluídos:
- Contagem de dados nulos
- Distribuição de variáveis com histogramas ou boxplots;
- Correlações entre variáveis com heatmaps;
- Identificação de outliers com gráficos de dispersão e boxplots; e
- Contagem e análise de transações fraudulentas vs. não fraudulentas

In [4]:
# Informações iniciais do dataset
dados_amostra.info()

<class 'pandas.core.frame.DataFrame'>
Index: 295270 entries, 470624 to 159120
Columns: 394 entries, TransactionID to V339
dtypes: float64(376), int64(4), object(14)
memory usage: 889.8+ MB


In [5]:
# Análise estatística descritiva
print("Tipos de dados das features:")
display(dados_amostra.dtypes)

Tipos de dados das features:


,0
TransactionID,int64
isFraud,int64
TransactionDT,int64
TransactionAmt,float64
ProductCD,object
...,...
V335,float64
V336,float64
V337,float64
V338,float64


In [6]:
print("Descrição estatística das features numéricas:")
display(dados_amostra.describe())

Descrição estatística das features numéricas:


,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
count,2.952700e+05,295270.000000,2.952700e+05,295270.000000,295270.000000,290889.000000,294492.000000,293159.000000,262567.000000,262567.000000,...,41271.000000,41271.000000,41271.000000,41271.000000,41271.000000,41271.000000,41271.000000,41271.000000,41271.000000,41271.000000
mean,3.282229e+06,0.035344,7.371075e+06,134.846991,9888.016832,362.720302,153.187122,199.306963,290.482814,86.804610,...,0.793753,733.588658,1405.450575,1040.191781,10.522280,60.163951,29.363972,58.365184,156.191612,104.407490
std,1.704781e+05,0.184648,4.617285e+06,240.282201,4906.750649,157.804099,11.310968,41.234767,101.568664,2.652733,...,4.797616,6257.123378,11294.053907,8073.775494,293.944405,408.113355,319.883041,782.563760,1175.353500,911.499788
min,2.987000e+06,0.000000,8.640000e+04,0.251000,1000.000000,100.000000,100.000000,100.000000,100.000000,10.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.134697e+06,0.000000,3.027840e+06,42.950000,6019.000000,215.000000,150.000000,166.000000,204.000000,87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.282391e+06,0.000000,7.309347e+06,68.500000,9635.000000,361.000000,150.000000,226.000000,299.000000,87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.429456e+06,0.000000,1.123381e+07,124.970000,14182.000000,512.000000,150.000000,226.000000,330.000000,87.000000,...,0.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.577539e+06,1.000000,1.581113e+07,31937.391000,18396.000000,600.000000,231.000000,237.000000,540.000000,102.000000,...,55.000000,160000.000000,160000.000000,160000.000000,55125.000000,55125.000000,55125.000000,104060.000000,104060.000000,104060.000000


In [7]:
# Verificar e ordenar as colunas com mais valores nulos
nulos_por_coluna = dados_amostra.isnull().sum()

# Filtrar apenas as colunas com valores nulos e ordenar em ordem decrescente
colunas_com_mais_nulos = nulos_por_coluna[nulos_por_coluna > 0].sort_values(ascending=False)
pd.set_option('display.max_rows', None)  # Remove a limitação de linhas exibidas

##Etapa 4: Tratamento de Valores Ausentes
Nessa etapa estamos focados em identificar as colunas com valores ausentes e aplicar técnicas de preenchimento ou remoção, dependendo da proporção de dados ausentes. Use métodos como:
- Preenchimento com média, mediana ou moda
- Remoção de linhas/colunas com muitos valores ausentes

In [8]:
# Definir o limite para remover colunas com mais de 50% de valores nulos
limite = len(dados_amostra) * 0.5

# Remover colunas com mais de 50% de valores nulos
dados_sem_muitos_nulos = dados_amostra.dropna(thresh=limite, axis=1)

# Verificar se ainda restam valores nulos
print("\nVerificação de valores nulos após a exclusão de colunas:")
print(dados_sem_muitos_nulos.isnull().sum().sort_values(ascending=False).head(10))

# Remover qualquer linha que ainda tenha valores nulos
dados_sem_muitos_nulos.dropna(inplace=True)

# Verificar novamente os valores nulos
print("\nValores nulos restantes (após remoção de linhas com nulos):")
print(dados_sem_muitos_nulos.isnull().sum().sort_values(ascending=False).head(10))



Verificação de valores nulos após a exclusão de colunas:
M4     140764
D2     140118
V1     139516
V10    139516
D11    139516
V2     139516
V3     139516
V4     139516
V5     139516
V6     139516
dtype: int64

Valores nulos restantes (após remoção de linhas com nulos):
TransactionID    0
V112             0
V101             0
V102             0
V103             0
V104             0
V105             0
V106             0
V107             0
V108             0
dtype: int64


<ipython-input-8-3ee9f38b4e68>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_sem_muitos_nulos.dropna(inplace=True)


In [9]:
# Comparação do número de colunas antes e depois da remoção
info_comparacao = pd.DataFrame({
    'Status': ['Antes da Remoção', 'Depois da Remoção'],
    'Linhas': [dados_amostra.shape[0], dados_sem_muitos_nulos.shape[0]],
    'Colunas': [dados_amostra.shape[1], dados_sem_muitos_nulos.shape[1]]
})

# Exibir a tabela de comparação
display(info_comparacao)

,Status,Linhas,Colunas
0,Antes da Remoção,295270,394
1,Depois da Remoção,38318,220


##Etapa 5: Feature Engineering
Descrição: Realize a criação ou transformação de novas features que possam ser úteis para o modelo. Isso pode incluir:
- Transformação de variáveis categóricas em variáveis numéricas (one-hot encoding)
- Criação de variáveis temporais ou agregadas
- Seleção de variáveis mais relevantes através de métodos como feature importance ou análise de variância

### Transformação de Variáveis Categóricas (One-Hot Encoding)


In [10]:
# Verificar se existem colunas categóricas
colunas_categoricas = dados_amostra.select_dtypes(include=['object']).columns

# Aplicar One-Hot Encoding nas variáveis categóricas, se houver
if len(colunas_categoricas) > 0:
    dados_amostra = pd.get_dummies(dados_amostra, columns=colunas_categoricas, drop_first=True)
    print(f"Colunas categóricas transformadas em dummies: {colunas_categoricas}")
else:
    print("Não há colunas categóricas para transformar.")

Colunas categóricas transformadas em dummies: Index(['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1',
       'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9'],
      dtype='object')


- Identifica colunas categóricas (se houver).
- Transforma as colunas categóricas em colunas numéricas chamadas de dummies (variáveis que assumem valores binários, 0 ou 1).

### Criação de Variáveis temporais

In [11]:
# Exemplo: Criar uma nova feature como a razão entre duas colunas
if 'V1' in dados.columns and 'V2' in dados.columns:  # Verifique se as colunas existem
    dados_amostra['Razao_V1_V2'] = dados_amostra['V1'] / (dados_amostra['V2'] + 1e-9)  # Evitar divisão por zero
    print("Nova feature 'Razao_V1_V2' criada.")


Nova feature 'Razao_V1_V2' criada.


- Verifica se as colunas V1 e V2 existem no dataset.
- Cria uma nova variável (Razao_V1_V2) que é a divisão dos valores de V1 pelos valores de V2.

Criar novas variáveis a partir das existentes pode fornecer mais informações ao modelo e ajudar a detectar padrões.


## Etapa 6: Normalização/Padronização
Descrição: Aplique normalização ou padronização às features numéricas, o que é essencial para o desempenho do LSTM. Use métodos como:
- Min-Max Scaling
- Z-score (padronização)

In [12]:
# Escolher a técnica de normalização/padronização
scaler = MinMaxScaler()

# Aplicar a normalização nas colunas numéricas
colunas_numericas = dados.select_dtypes(include=['float64', 'int64']).columns
dados_amostra[colunas_numericas] = scaler.fit_transform(dados_amostra[colunas_numericas])

print("Normalização aplicada nas colunas numéricas.")

Normalização aplicada nas colunas numéricas.


- Seleciona todas as colunas numéricas do dataset.
- Normaliza os valores dessas colunas para que todos os valores fiquem entre 0 e 1, utilizando o Min-Max Scaling.

In [13]:
# Escolher a técnica de normalização (Min-Max Scaling)
scaler = MinMaxScaler()

# Selecionar todas as colunas numéricas que foram limpas (sem nulos)
colunas_numericas = dados_sem_muitos_nulos.select_dtypes(include=['float64', 'int64']).columns

# Aplicar a normalização nas colunas numéricas
dados_sem_muitos_nulos[colunas_numericas] = scaler.fit_transform(dados_sem_muitos_nulos[colunas_numericas])

# Salvando o dataset processado até a etapa 6
# dados_sem_muitos_nulos.to_csv('/content/drive/MyDrive/Colab Notebooks/dados_sem_muitos_nulos_normalizados.csv', index=False)


- **Seleção das colunas numéricas limpas**: O código identifica as colunas numéricas que foram limpas na Etapa 4 (sem valores nulos).
- **Aplicação do Min-Max Scaling**: A normalização é aplicada a essas colunas, ajustando os valores para uma faixa entre 0 e 1.
- **Exibição dos dados normalizados**: Após a normalização, as primeiras linhas das colunas numéricas são exibidas para visualização.

## Etapa EXTRA: Criação de Sequências Temporais
Descrição: Prepare os dados no formato apropriado para o LSTM, criando sequências temporais. Isso pode incluir:
- Dividir os dados em janelas temporais
- Ajustar os dados de entrada para atender ao formato (samples, timesteps, features)


Essa etapa (Criação de Sequências Temporais) é importante de incluir porque o modelo LSTM (Long Short-Term Memory) é projetado para capturar dependências temporais entre os dados. A principal característica do LSTM é sua habilidade de aprender e fazer previsões com base em padrões de sequências temporais. Assim estou captrando padrões temporais e tentando fazer a rede lembrar de informações passadas, no caso com
```
window_size
```



In [14]:
# Definir função para criar sequências temporais
def criar_sequencias(dados, target, window_size):
    sequencias = []
    alvos = []

    for i in range(len(dados) - window_size):
        sequencia = dados[i:i + window_size]  # Cria a janela com o tamanho especificado
        alvo = target[i + window_size]        # O valor a ser previsto vem após a janela
        sequencias.append(sequencia)
        alvos.append(alvo)

    return np.array(sequencias), np.array(alvos)

## Etapa 7: Modelo LSTM

In [15]:
# Normalizar os dados e preparar para treinamento
scaler = MinMaxScaler()

# Aplicar a transformação de variáveis categóricas
colunas_categoricas = dados_sem_muitos_nulos.select_dtypes(include=['object']).columns
dados_transformados = pd.get_dummies(dados_sem_muitos_nulos, columns=colunas_categoricas, drop_first=True)

# Definir X e y (usando a coluna 'isFraud' como target)
X = dados_transformados.drop('isFraud', axis=1).values
y = dados_transformados['isFraud'].values

# Normalizar os dados
X = scaler.fit_transform(X)

# Definir o tamanho da janela (número de timesteps)
window_size = 5  # Exemplo: 5 transações anteriores

# Criar as sequências temporais
X_sequencias, y_sequencias = criar_sequencias(X, y, window_size)

# Dividir em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_sequencias, y_sequencias, test_size=0.2, random_state=42)

# **Definir o Modelo Original (Sem Ajustes)**:
model_original = Sequential()
model_original.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
model_original.add(Dense(32, activation='relu'))
model_original.add(Dense(1, activation='sigmoid'))  # Saída binária (fraude ou não)

# Compilar o modelo
model_original.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
history_original = model_original.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Avaliar o modelo no conjunto de teste
loss_original, accuracy_original = model_original.evaluate(X_test, y_test)
print(f"Test Accuracy (Modelo Original): {accuracy_original:.4f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - accuracy: 0.9619 - loss: 0.1740 - val_accuracy: 0.9646 - val_loss: 0.1531
Epoch 2/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - accuracy: 0.9603 - loss: 0.1675 - val_accuracy: 0.9646 - val_loss: 0.1530
Epoch 3/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9621 - loss: 0.1613 - val_accuracy: 0.9646 - val_loss: 0.1533
Epoch 4/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9642 - loss: 0.1540 - val_accuracy: 0.9646 - val_loss: 0.1538
Epoch 5/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9646 - loss: 0.1518 - val_accuracy: 0.9646 - val_loss: 0.1547
Epoch 6/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - accuracy: 0.9630 - loss: 0.1570 - val_accuracy: 0.9646 - val_loss: 0.1558
Epoch 7/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9624 - loss: 0.1579 - val_accuracy: 0.9646 - val_loss: 0.1552
Epoch 8/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9624 - loss: 0.1584 - val_ac

### Etapa 7.1.1: Definição da Arquitetura do Modelo LSTM Original
Descrição: Defina a estrutura da rede LSTM, escolhendo:
- Número de camadas LSTM
- Número de neurônios por camada
- Função de ativação
- Dropout (se necessário)
- Hiperparâmetros de otimização (otimizador, taxa de aprendizado, etc.)
- Função de perda e métricas


In [16]:
# Definir a arquitetura do modelo LSTM
model_original = Sequential()

# Adicionar uma camada LSTM
model_original.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))

# Adicionar Dropout para regularização (evitar overfitting)
model_original.add(Dropout(0.2))

# Adicionar uma camada densa com 32 neurônios
model_original.add(Dense(32, activation='relu'))

# Camada de saída com ativação sigmoid para prever fraude ou não
model_original.add(Dense(1, activation='sigmoid'))

# Compilar o model_original
model_original.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Resumo do model_original
model_original.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                        │ (None, 64)                  │          83,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 85,569 (334.25 KB)

 Trainable params: 85,569 (334.25 KB)

 Non-trainable params: 0 (0.00 B)

### Etapa 7.1.2: Divisão do Conjunto de Dados (Treinamento/Validação/Teste)
Descrição: Divida os dados em conjunto de treinamento, validação e teste. Certifique-se de que a divisão preserve a sequência temporal.

1. Treinar o modelo LSTM com os dados de treinamento que criamos.
2. Validar o modelo em um conjunto de dados separado (dados de teste) para garantir que ele não está superajustado (overfitting) aos dados de treinamento.
3. Avaliar o desempenho usando métricas como precisão, recall, F1-score, e AUC-ROC.

In [17]:
# Treinar o modelo LSTM
history = model_original.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Avaliar o modelo no conjunto de teste
loss, accuracy = model_original.evaluate(X_test, y_test)
display(f"Test Accuracy: {accuracy:.4f}")

Epoch 1/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9618 - loss: 0.1751 - val_accuracy: 0.9646 - val_loss: 0.1529
Epoch 2/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9641 - loss: 0.1569 - val_accuracy: 0.9646 - val_loss: 0.1530
Epoch 3/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9602 - loss: 0.1694 - val_accuracy: 0.9646 - val_loss: 0.1533
Epoch 4/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.9612 - loss: 0.1657 - val_accuracy: 0.9646 - val_loss: 0.1556
Epoch 5/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.9630 - loss: 0.1585 - val_accuracy: 0.9646 - val_loss: 0.1546
Epoch 6/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - accuracy: 0.9620 - loss: 0.1609 - val_accuracy: 0.9646 - val_loss: 0.1582
Epoch 7/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9622 - loss: 0.1604 - val_accuracy: 0.9646 - val_loss: 0.1546
Epoch 8/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 0.9618 - loss: 0.1610 - val_accurac

'Test Accuracy: 0.9645'

In [18]:
# Calcular os pesos para cada classe
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

# Verificar os pesos calculados
print(f"Class Weights: {class_weight_dict}")

# Ajustar o modelo original ou original, adicionando o parâmetro class_weight no fit
history = model_original.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), class_weight=class_weight_dict)

# Avaliar o modelo original
loss_original, accuracy_original = model_original.evaluate(X_test, y_test)
print(f"Test Accuracy (Modelo original com Pesos): {accuracy_original:.4f}")


Class Weights: {0: 0.5195972062114328, 1: 13.256920415224913}
Epoch 1/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.7530 - loss: 0.6932 - val_accuracy: 0.7034 - val_loss: 0.5632
Epoch 2/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.6925 - loss: 0.5930 - val_accuracy: 0.6761 - val_loss: 0.5687
Epoch 3/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.6832 - loss: 0.5702 - val_accuracy: 0.6525 - val_loss: 0.6324
Epoch 4/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.7302 - loss: 0.5287 - val_accuracy: 0.7642 - val_loss: 0.4716
Epoch 5/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.7166 - loss: 0.5197 - val_accuracy: 0.6479 - val_loss: 0.6111
Epoch 6/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.7378 - loss: 0.4842 - val_accuracy: 0.7376 - val_loss: 0.5076
Epoch 7/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 0.7459 - loss: 0.4701 - val_accuracy: 0.7042 - val_loss: 0.5592
Epoch 8/20
958/958 ━━━━━━━━━━━━━━━━

### Etapa 7.2 (Modelo Ajustado - Com Ajustes):


In [19]:
# **Definir o Modelo Ajustado (Com Dropout e LR reduzido)**:
model_ajustado = Sequential()
model_ajustado.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))

# Adicionar Dropout para regularização (evitar overfitting)
model_ajustado.add(Dropout(0.5))

# Adicionar uma camada densa com 32 neurônios
model_ajustado.add(Dense(32, activation='relu'))
model_ajustado.add(Dropout(0.5))  # Adicionar Dropout após a camada densa

# Camada de saída com ativação sigmoid para prever fraude ou não
model_ajustado.add(Dense(1, activation='sigmoid'))

# Compilar o modelo com uma taxa de aprendizado menor
model_ajustado.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo ajustado
history_ajustado = model_ajustado.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Avaliar o modelo no conjunto de teste
loss_ajustado, accuracy_ajustado = model_ajustado.evaluate(X_test, y_test)
print(f"Test Accuracy (Modelo Ajustado): {accuracy_ajustado:.4f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - accuracy: 0.9454 - loss: 0.2211 - val_accuracy: 0.9646 - val_loss: 0.1531
Epoch 2/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9631 - loss: 0.1779 - val_accuracy: 0.9646 - val_loss: 0.1531
Epoch 3/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9620 - loss: 0.1761 - val_accuracy: 0.9646 - val_loss: 0.1531
Epoch 4/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9633 - loss: 0.1725 - val_accuracy: 0.9646 - val_loss: 0.1534
Epoch 5/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 0.9627 - loss: 0.1742 - val_accuracy: 0.9646 - val_loss: 0.1533
Epoch 6/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - accuracy: 0.9629 - loss: 0.1703 - val_accuracy: 0.9646 - val_loss: 0.1534
Epoch 7/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9634 - loss: 0.1683 - val_accuracy: 0.9646 - val_loss: 0.1536
Epoch 8/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.9626 - loss: 0.1684 - val_accur

### Etapa 7.2.1: Definição da Arquitetura do Modelo LSTM
Descrição: Defina a estrutura da rede LSTM, escolhendo:
- Número de camadas LSTM
- Número de neurônios por camada
- Função de ativação
- Dropout (se necessário)
- Hiperparâmetros de otimização (otimizador, taxa de aprendizado, etc.)
- Função de perda e métricas


In [20]:
# Definir a arquitetura do model_ajustadoo LSTM
model_ajustado = Sequential()

# Adicionar uma camada LSTM
model_ajustado.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))

# Adicionar Dropout para regularização (evitar overfitting)
model_ajustado.add(Dropout(0.2))

# Adicionar uma camada densa com 32 neurônios
model_ajustado.add(Dense(32, activation='relu'))

# Camada de saída com ativação sigmoid para prever fraude ou não
model_ajustado.add(Dense(1, activation='sigmoid'))

# Compilar o model_ajustado
model_ajustado.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
# Resumo do model_ajustado
model_ajustado.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 64)                  │          83,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 85,569 (334.25 KB)

 Trainable params: 85,569 (334.25 KB)

 Non-trainable params: 0 (0.00 B)

### Etapa 7.2.2: Divisão do Conjunto de Dados (Treinamento/Validação/Teste)
Descrição: Divida os dados em conjunto de treinamento, validação e teste. Certifique-se de que a divisão preserve a sequência temporal.

1. Treinar o modelo LSTM com os dados de treinamento que criamos.
2. Validar o modelo em um conjunto de dados separado (dados de teste) para garantir que ele não está superajustado (overfitting) aos dados de treinamento.
3. Avaliar o desempenho usando métricas como precisão, recall, F1-score, e AUC-ROC.
4. Monitorar o treinamento através das curvas de aprendizado para verificar possíveis sinais de overfitting ou underfitting.

1. Treinar o modelo LSTM com os dados de treinamento que criamos.

In [21]:
# Treinar o modelo LSTM
history = model_ajustado.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Avaliar o modelo no conjunto de teste
loss, accuracy = model_ajustado.evaluate(X_test, y_test)
display(f"Test Accuracy: {accuracy:.4f}")

Epoch 1/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 13s 12ms/step - accuracy: 0.9522 - loss: 0.1849 - val_accuracy: 0.9646 - val_loss: 0.1538
Epoch 2/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 14s 14ms/step - accuracy: 0.9632 - loss: 0.1597 - val_accuracy: 0.9646 - val_loss: 0.1544
Epoch 3/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.9622 - loss: 0.1628 - val_accuracy: 0.9646 - val_loss: 0.1539
Epoch 4/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - accuracy: 0.9631 - loss: 0.1588 - val_accuracy: 0.9646 - val_loss: 0.1551
Epoch 5/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.9627 - loss: 0.1589 - val_accuracy: 0.9646 - val_loss: 0.1563
Epoch 6/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.9611 - loss: 0.1648 - val_accuracy: 0.9646 - val_loss: 0.1545
Epoch 7/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - accuracy: 0.9634 - loss: 0.1575 - val_accuracy: 0.9646 - val_loss: 0.1550
Epoch 8/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.9622 - loss: 0.1597 - val

'Test Accuracy: 0.9645'

In [22]:
from sklearn.metrics import classification_report, roc_auc_score

# Fazer previsões com o modelo
y_pred = (model_ajustado.predict(X_test) > 0.5).astype(int)

# Relatório de classificação com precisão, recall e F1-score
print(classification_report(y_test, y_pred))

# Calcular e exibir o AUC-ROC
auc = roc_auc_score(y_test, y_pred)
print(f"AUC-ROC: {auc:.4f}")

240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      7392
         1.0       0.00      0.00      0.00       271

    accuracy                           0.96      7663
   macro avg       0.48      0.50      0.49      7663
weighted avg       0.93      0.96      0.95      7663

AUC-ROC: 0.4999


3. Avaliar o desempenho usando métricas como precisão, recall, F1-score, e AUC-ROC.

In [23]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Calcular os pesos para cada classe
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

# Verificar os pesos calculados
print(f"Class Weights: {class_weight_dict}")

# Ajustar o modelo original ou ajustado, adicionando o parâmetro class_weight no fit
history = model_ajustado.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), class_weight=class_weight_dict)

# Avaliar o modelo ajustado
loss_ajustado, accuracy_ajustado = model_ajustado.evaluate(X_test, y_test)
print(f"Test Accuracy (Modelo Ajustado com Pesos): {accuracy_ajustado:.4f}")


Class Weights: {0: 0.5195972062114328, 1: 13.256920415224913}
Epoch 1/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.6505 - loss: 0.6653 - val_accuracy: 0.4286 - val_loss: 0.8210
Epoch 2/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.6847 - loss: 0.6000 - val_accuracy: 0.5922 - val_loss: 0.6663
Epoch 3/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.6917 - loss: 0.5714 - val_accuracy: 0.6555 - val_loss: 0.5886
Epoch 4/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.7009 - loss: 0.5486 - val_accuracy: 0.7038 - val_loss: 0.5526
Epoch 5/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7208 - loss: 0.5052 - val_accuracy: 0.5255 - val_loss: 0.7781
Epoch 6/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.6898 - loss: 0.5144 - val_accuracy: 0.7403 - val_loss: 0.4943
Epoch 7/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.7560 - loss: 0.4471 - val_accuracy: 0.7069 - val_loss: 0.5504
Epoch 8/20
958/958 ━━━━━━━━━━━━━━━━━━━

## Etapa 8: Aplicação do SMOTE para melhorar o modelo

In [25]:
from imblearn.over_sampling import SMOTE

# Aplicar SMOTE para balancear o dataset de treino
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)

# Treinar o modelo ajustado novamente com o dataset balanceado
history_smote = model_ajustado.fit(X_train_res.reshape(X_train_res.shape[0], window_size, X_train.shape[2]), y_train_res, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Avaliar o modelo ajustado
loss_smote, accuracy_smote = model_ajustado.evaluate(X_test, y_test)
print(f"Test Accuracy (Modelo Ajustado com SMOTE): {accuracy_smote:.4f}")

Epoch 1/20
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 26s 14ms/step - accuracy: 0.9292 - loss: 0.1913 - val_accuracy: 0.8794 - val_loss: 0.5630
Epoch 2/20
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 31s 8ms/step - accuracy: 0.9574 - loss: 0.1246 - val_accuracy: 0.9010 - val_loss: 0.5547
Epoch 3/20
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - accuracy: 0.9700 - loss: 0.0885 - val_accuracy: 0.9143 - val_loss: 0.6369
Epoch 4/20
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9789 - loss: 0.0656 - val_accuracy: 0.9011 - val_loss: 0.6758
Epoch 5/20
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - accuracy: 0.9824 - loss: 0.0512 - val_accuracy: 0.9345 - val_loss: 0.6856
Epoch 6/20
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - accuracy: 0.9857 - loss: 0.0425 - val_accuracy: 0.9225 - val_loss: 0.7350
Epoch 7/20
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.9893 - loss: 0.0329 - val_accuracy: 0.9294 - val_loss: 0.7727
Epoch 8/20
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 24s 10ms/step - accuracy: 0.9909 - loss:

In [26]:
from imblearn.over_sampling import SMOTE

# Aplicar SMOTE para balancear o dataset de treino
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)

# Treinar o modelo ajustado novamente com o dataset balanceado
history_smote = model_original.fit(X_train_res.reshape(X_train_res.shape[0], window_size, X_train.shape[2]), y_train_res, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Avaliar o modelo ajustado
loss_smote, accuracy_smote = model_original.evaluate(X_test, y_test)
print(f"Test Accuracy (Modelo Ajustado com SMOTE): {accuracy_smote:.4f}")

Epoch 1/20
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.9274 - loss: 0.1956 - val_accuracy: 0.8605 - val_loss: 0.5619
Epoch 2/20
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.9554 - loss: 0.1279 - val_accuracy: 0.8900 - val_loss: 0.5947
Epoch 3/20
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.9671 - loss: 0.0963 - val_accuracy: 0.8878 - val_loss: 0.6808
Epoch 4/20
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.9769 - loss: 0.0694 - val_accuracy: 0.9277 - val_loss: 0.6757
Epoch 5/20
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.9828 - loss: 0.0520 - val_accuracy: 0.9263 - val_loss: 0.7326
Epoch 6/20
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.9857 - loss: 0.0443 - val_accuracy: 0.9050 - val_loss: 0.7878
Epoch 7/20
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.9877 - loss: 0.0357 - val_accuracy: 0.9273 - val_loss: 0.7610
Epoch 8/20
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.9910 - loss: 0

In [28]:
from sklearn.metrics import classification_report, roc_auc_score
import plotly.graph_objects as go
import pandas as pd

# Verificar se a classe '1' existe no relatório de classificação
def safe_get_metric(report, label, metric):
    try:
        return report[label][metric]
    except KeyError:
        return 0.0  # Retorna 0.0 se a classe '1' não for encontrada no relatório

# Fazer previsões com o modelo original
y_pred_original = (model_original.predict(X_test) > 0.5).astype(int)
print("Relatório de Classificação - Modelo Original")
report_original = classification_report(y_test, y_pred_original, output_dict=True)
print(classification_report(y_test, y_pred_original))
auc_original = roc_auc_score(y_test, y_pred_original)
print(f"AUC-ROC Modelo Original: {auc_original:.4f}")

# Fazer previsões com o modelo ajustado
y_pred_ajustado = (model_ajustado.predict(X_test) > 0.5).astype(int)
print("Relatório de Classificação - Modelo Ajustado")
report_ajustado = classification_report(y_test, y_pred_ajustado, output_dict=True)
print(classification_report(y_test, y_pred_ajustado))
auc_ajustado = roc_auc_score(y_test, y_pred_ajustado)
print(f"AUC-ROC Modelo Ajustado: {auc_ajustado:.4f}")

# Comparar as métricas em um DataFrame
metrics_comparison = pd.DataFrame({
    'Modelo Original': {
        'Precision': safe_get_metric(report_original, '1', 'precision'),
        'Recall': safe_get_metric(report_original, '1', 'recall'),
        'F1-Score': safe_get_metric(report_original, '1', 'f1-score'),
        'AUC-ROC': auc_original
    },
    'Modelo Ajustado': {
        'Precision': safe_get_metric(report_ajustado, '1', 'precision'),
        'Recall': safe_get_metric(report_ajustado, '1', 'recall'),
        'F1-Score': safe_get_metric(report_ajustado, '1', 'f1-score'),
        'AUC-ROC': auc_ajustado
    }
})

# Exibir a comparação
print(metrics_comparison)


240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Relatório de Classificação - Modelo Original
              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97      7392
         1.0       0.03      0.02      0.03       271

    accuracy                           0.94      7663
   macro avg       0.50      0.50      0.50      7663
weighted avg       0.93      0.94      0.94      7663

AUC-ROC Modelo Original: 0.4993
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Relatório de Classificação - Modelo Ajustado
              precision    recall  f1-score   support

         0.0       0.96      0.97      0.97      7392
         1.0       0.04      0.03      0.03       271

    accuracy                           0.94      7663
   macro avg       0.50      0.50      0.50      7663
weighted avg       0.93      0.94      0.93      7663

AUC-ROC Modelo Ajustado: 0.5001
           Modelo Original  Modelo Ajustado
Precision         0.000000          0.00000
Recall            0.00

In [31]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Calcular os pesos para cada classe
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

# Treinar o modelo ajustado com os pesos das classes
history = model_ajustado.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), class_weight=class_weight_dict)

# Avaliar o modelo ajustado
loss_ajustado, accuracy_ajustado = model_ajustado.evaluate(X_test, y_test)
print(f"Test Accuracy (Modelo Ajustado com Pesos): {accuracy_ajustado:.4f}")


Epoch 1/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - accuracy: 0.9884 - loss: 0.0271 - val_accuracy: 0.9047 - val_loss: 0.6864
Epoch 2/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - accuracy: 0.9887 - loss: 0.0274 - val_accuracy: 0.9209 - val_loss: 0.6369
Epoch 3/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.9832 - loss: 0.0431 - val_accuracy: 0.9385 - val_loss: 0.6804
Epoch 4/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9881 - loss: 0.0345 - val_accuracy: 0.8829 - val_loss: 0.6610
Epoch 5/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.9806 - loss: 0.0482 - val_accuracy: 0.9398 - val_loss: 0.6957
Epoch 6/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9913 - loss: 0.0243 - val_accuracy: 0.9002 - val_loss: 0.5837
Epoch 7/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9897 - loss: 0.0244 - val_accuracy: 0.8191 - val_loss: 0.8833
Epoch 8/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9644 - loss: 0.0894 - val_acc

In [33]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Calcular os pesos para cada classe
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

# Treinar o modelo ajustado com os pesos das classes
history = model_original.fit(X_train, y_train, epochs=20
                             , batch_size=32, validation_data=(X_test, y_test), class_weight=class_weight_dict)

# Avaliar o modelo ajustado
loss_ajustado, accuracy_ajustado = model_original.evaluate(X_test, y_test)
print(f"Test Accuracy (Modelo Original com Pesos): {accuracy_ajustado:.4f}")


Epoch 1/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.9879 - loss: 0.0268 - val_accuracy: 0.8961 - val_loss: 0.7992
Epoch 2/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9857 - loss: 0.0313 - val_accuracy: 0.8293 - val_loss: 0.7984
Epoch 3/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 0.9606 - loss: 0.0889 - val_accuracy: 0.9097 - val_loss: 0.7049
Epoch 4/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9880 - loss: 0.0312 - val_accuracy: 0.9246 - val_loss: 0.6670
Epoch 5/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9830 - loss: 0.0461 - val_accuracy: 0.8506 - val_loss: 0.7817
Epoch 6/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9864 - loss: 0.0255 - val_accuracy: 0.9161 - val_loss: 0.7401
Epoch 7/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9908 - loss: 0.0221 - val_accuracy: 0.9154 - val_loss: 0.5972
Epoch 8/20
958/958 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.9857 - loss: 0.0335 - val_accurac

Esses gráficos indicam sinais claros de overfitting. Isso significa que o modelo tem um desempenho muito bom no conjunto de dados de treinamento, mas não generaliza bem para os dados de validação. Isso pode ser um problema em modelos de machine learning, pois significa que o modelo pode não funcionar bem em novos dados.


## Etapa 14: Análise e Discussão dos Resultados
Descrição: Compare o desempenho do modelo ao longo do treinamento e teste. Discuta os principais insights obtidos com o modelo e estratégias usadas para melhorar a performance.

### Modelo Ajustado (com pesos nas classes):
- Acurácia de Treinamento: Variou ao longo das épocas, com uma média alta, acima de 98%, o que indica que o modelo está aprendendo bem no conjunto de treinamento.
- Acurácia de Validação: Variou entre 88% a 94% em diferentes épocas, mas foi consistente, com um valor final de 93.17%.
- Perda de Validação: Apesar da acurácia relativamente alta, a perda de validação ainda é alta (em torno de 0.68), o que indica que o modelo está aprendendo, mas não de forma perfeita.

### Modelo Original (com pesos nas classes):
- Acurácia de Treinamento: Também foi alta, geralmente acima de 98%, com pequenas flutuações.
- Acurácia de Validação: Variou significativamente mais que o modelo ajustado, ficando entre 82% a 93%, indicando que o modelo pode estar tendo mais dificuldade em generalizar bem do que o modelo ajustado.
- Perda de Validação: A perda foi consistente, mas também alta (em torno de 0.68).


## Conclusões e Comparações:
- Acurácia: Ambos os modelos têm uma acurácia final de 93.17%, o que é um bom resultado considerando o desbalanceamento de classes.

- Perda de Validação: Ambos os modelos têm uma perda relativamente alta na validação, o que indica que ainda há espaço para melhorias. A perda de validação no modelo ajustado foi 0.6655, enquanto a do modelo original foi 0.6814. A diferença é pequena, mas o modelo ajustado teve um desempenho um pouco melhor.

- Validação Flutuante: O modelo original pareceu ter mais dificuldade com variações na validação, especialmente nas primeiras épocas, onde houve mais oscilações na acurácia e perda.

- Recomendações para Melhorias:
Reduzir o Overfitting:

- As variações nas perdas e acurácias sugerem que pode haver overfitting. Para melhorar isso, você pode tentar técnicas como:
  - Aumentar o Dropout.
  - Implementar Early Stopping de forma mais agressiva.
  - Adicionar Regularização L2 às camadas densas para penalizar pesos altos.

- Balanceamento dos Dados:
Apesar de ter usado pesos para balancear as classes, o desbalanceamento parece ainda estar impactando a detecção da classe minoritária (fraude). O uso do SMOTE para criar mais amostras da classe de fraude pode ajudar a aumentar a detecção de fraudes.

- Ajuste do Threshold de Classificação:
Como as fraudes são mais raras do que a normalidade, o modelo pode estar classificando muitas transações como "não fraude". Ajustar o threshold para prever fraudes com mais confiança (usando um valor de threshold inferior a 0.5, como 0.3 ou 0.2) pode melhorar o recall para fraudes.

- Monitoramento da Curva de Aprendizado:
Monitorar a curva de perda e acurácia ao longo das épocas, aplicando early stopping quando a perda de validação começa a aumentar consistentemente, pode evitar que o modelo continue treinando além do necessário.

### Conclusão:
Embora o desempenho do modelo ajustado com pesos tenha sido ligeiramente melhor, ambos os modelos têm espaço para melhoria, especialmente para detectar fraudes. Experimente usar o SMOTE, ajustes de threshold, e talvez mais regularização para tentar melhorar a detecção de fraudes.